# Titanic 분류

In [62]:
# 파일 읽어오기
import pandas as pd

titanic = pd.read_csv('titanic_trainset.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [63]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


종속변수 : Survived  
필요없는 변수 : PassengerId, Name, Ticket,  
애매 : Cabin

In [64]:
# 필요없는 변수 삭제 
titanic_case1 = titanic.drop(['PassengerId','Name','Ticket'],axis=1)        # inplace=True 로 지정하면 다시 변수 안줘도 됨.
del titanic_case1['Cabin']
titanic_case1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [65]:
titanic_case1.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

Age는 중위값, Embarked는 최빈값으로 대체  

In [66]:
titanic_case1['Age'].fillna(titanic_case1['Age'].median(), inplace = True)
titanic_case1['Embarked'].fillna(titanic_case1['Embarked'].mode()[0], inplace = True)

titanic_case1.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [50]:
titanic_x = titanic_case1.iloc[:,1:]
titanic_y = titanic_case1.iloc[:,0]

In [7]:
titanic_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [51]:
from sklearn.preprocessing import LabelEncoder

titanic_x['Sex'] = LabelEncoder().fit_transform(titanic_x['Sex'])
titanic_x['Embarked'] = LabelEncoder().fit_transform(titanic_x['Embarked'])

'''
이렇게도 가능 
features = ["Sex","Embarked"]
for feature in features:
    titanic[feature] = LabelEncoder().fit_transform(titanic[feature])
'''

titanic_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2


In [52]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

encoder = OneHotEncoder()
embarked = encoder.fit_transform(np.array(titanic_x['Embarked']).reshape(-1,1))
embarked.toarray()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [53]:
from sklearn.preprocessing import MinMaxScaler

titanic_x = MinMaxScaler().fit_transform(titanic_x)

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(titanic_x, titanic_y, test_size=0.2,random_state=2021)

1) Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

dt= DecisionTreeClassifier(random_state=2021)

In [13]:
params = {
    'max_depth' : [x for x in range(2,8)],
    'min_samples_split': [2,3,4,5]
}

In [14]:
from sklearn.model_selection import GridSearchCV

grid_dt = GridSearchCV(dt,param_grid=params,cv=5,scoring='accuracy')
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3, 4, 5]},
             scoring='accuracy')

In [15]:
grid_dt.best_params_

{'max_depth': 4, 'min_samples_split': 2}

In [16]:
grid_dt.best_estimator_.score(X_test,y_test)

0.7653631284916201

In [17]:
estimator = grid_dt.best_estimator_

In [18]:
# 모델 평가
from sklearn.metrics import accuracy_score

pred_dt = estimator.predict(X_test)
accuracy_score(y_test,pred_dt)

0.7653631284916201

In [ ]:
from sklearn.model_selection import cross_val_score


2) 로지스틱

In [19]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=2021)

In [20]:
lr.fit(X_train, y_train)

LogisticRegression(random_state=2021)

In [21]:
pred_lr = lr.predict(X_test)

In [22]:
accuracy_score(y_test,pred_lr)

0.7486033519553073

3) Random Forest Classifier

In [55]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=2021)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=2021)

In [56]:
pred_rf = rf.predict(X_test)

accuracy_score(y_test, pred_rf)

0.7821229050279329

________________________________________________________

In [23]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
titanic_case2 = titanic.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
titanic_case2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [25]:
titanic_case2.pivot_table("Age","Sex","Pclass")

Pclass,1,2,3
Sex,,,
female,34.611765,28.722973,21.750000
male,41.281386,30.740707,26.507589


In [26]:
titanic_case2[titanic_case2['Age'].isnull()]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,0,3,male,NaN,0,0,8.4583,Q
17,1,2,male,NaN,0,0,13.0000,S
19,1,3,female,NaN,0,0,7.2250,C
26,0,3,male,NaN,0,0,7.2250,C
28,1,3,female,NaN,0,0,7.8792,Q
...,...,...,...,...,...,...,...,...
859,0,3,male,NaN,0,0,7.2292,C
863,0,3,female,NaN,8,2,69.5500,S
868,0,3,male,NaN,0,0,9.5000,S
878,0,3,male,NaN,0,0,7.8958,S


In [27]:
for i in range(889):
    if titanic_case2['Pclass'][i] == 1 :
        if titanic_case2['Sex'][i] == 'female' :
            if titanic_case2['Age'].isnull()[i]:
                titanic_case2['Age'][i] = 35
        else:
            if titanic_case2['Age'].isnull()[i]:
                titanic_case2['Age'][i] = 41

    elif titanic_case2['Pclass'][i] == 2:
        if titanic_case2['Sex'][i] == 'female' :
            if titanic_case2['Age'].isnull()[i]:
                titanic_case2['Age'][i] = 29
        else:
            if titanic_case2['Age'].isnull()[i]:
                titanic_case2['Age'][i] = 31
    else :
        if titanic_case2['Sex'][i] == 'female' :
            if titanic_case2['Age'].isnull()[i]:
                titanic_case2['Age'][i] = 22
        else:
            if titanic_case2['Age'].isnull()[i]:
                titanic_case2['Age'][i] = 27
        

<ipython-input-27-98ddf1221c49>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_case2['Age'][i] = 27
<ipython-input-27-98ddf1221c49>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_case2['Age'][i] = 31
<ipython-input-27-98ddf1221c49>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  titanic_case2['Age'][i] = 22
<ipython-input-27-98ddf1221c49>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [28]:
titanic_case2.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,27.0,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [29]:
titanic_case2['Embarked'].fillna(titanic_case2['Embarked'].mode()[0], inplace = True)

In [30]:
titanic_case2.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [31]:
from sklearn.preprocessing import LabelEncoder

titanic_case2['Sex'] = LabelEncoder().fit_transform(titanic_case2['Sex'])

In [32]:
from sklearn.preprocessing import OneHotEncoder

Embarked = OneHotEncoder().fit_transform(np.array(titanic_case2['Embarked']).reshape(-1,1)).toarray()
Embarked

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [33]:
titanic_case2.drop(['Embarked'],axis=1,inplace=True)

In [34]:
titanic_case2[['Embarked_C','Embarked_Q','Embarked_S']] = pd.DataFrame(Embarked, index = titanic_case2.index)
titanic_case2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0.0,0.0,1.0
1,1,1,0,38.0,1,0,71.2833,1.0,0.0,0.0
2,1,3,0,26.0,0,0,7.9250,0.0,0.0,1.0
3,1,1,0,35.0,1,0,53.1000,0.0,0.0,1.0
4,0,3,1,35.0,0,0,8.0500,0.0,0.0,1.0


In [35]:
titanic_x = titanic_case2.iloc[:,1:]
titanic_y = titanic_case2.iloc[:,0]

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(titanic_x, titanic_y, test_size=0.2,random_state=2021)

1) Decision Tree

In [37]:
from sklearn.tree import DecisionTreeClassifier

dt= DecisionTreeClassifier(random_state=2021)

In [38]:
params = {
    'max_depth' : [x for x in range(2,8)],
    'min_samples_split': [2,3,4,5]
}

In [39]:
from sklearn.model_selection import GridSearchCV

grid_dt = GridSearchCV(dt,param_grid=params,cv=5,scoring='accuracy')
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7],
                         'min_samples_split': [2, 3, 4, 5]},
             scoring='accuracy')

In [40]:
grid_dt.best_params_

{'max_depth': 3, 'min_samples_split': 2}

In [41]:
# 모델 평가

estimator = grid_dt.best_estimator_

from sklearn.metrics import accuracy_score

pred_dt = estimator.predict(X_test)
accuracy_score(y_test,pred_dt)

0.7541899441340782

2) SVM

In [42]:
from sklearn.svm import SVC

sv = SVC(random_state=2021)

sv_params = {
    'C' : [5,10,100],
    'kernel' : ['linear','rbf','sigmoid'],
    'gamma' : [5,10,100]
}

grid_sv = GridSearchCV(sv,param_grid=sv_params, cv=3)


GridSearchCV(cv=3, estimator=SVC(random_state=2021),
             param_grid={'C': [5, 10, 100], 'gamma': [5, 10, 100],
                         'kernel': ['linear', 'rbf', 'sigmoid']})

In [44]:
grid_sv.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=SVC(random_state=2021),
             param_grid={'C': [5, 10, 100], 'gamma': [5, 10, 100],
                         'kernel': ['linear', 'rbf', 'sigmoid']})

In [45]:
grid_sv.best_params_

{'C': 100, 'gamma': 5, 'kernel': 'linear'}

In [47]:
pred_sv = grid_sv.predict(X_test)

In [49]:
accuracy_score(y_test,pred_sv)

0.7206703910614525